In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute

G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3])
G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 0)])
nx.draw(G, with_labels=True, alpha=0.8, node_size=500)

In [ ]:
create_qaoa_circ(G, [1.0, 1.0]).draw(filename="../latex/img/circuits/qiskit_circuit_p-1_1.0.png")
create_qaoa_circ(G, [2.0, 2.0]).draw(filename="../latex/img/circuits/qiskit_circuit_p-1_2.0.png")
create_qaoa_circ(G, [1.0, 1.0, 1.0, 1.0]).draw(filename="../latex/img/circuits/qiskit_circuit_p-2_1.0.png")
create_qaoa_circ(G, [2.0, 2.0, 2.0, 2.0]).draw(filename="../latex/img/circuits/qiskit_circuit_p-2_2.0.png")
create_qaoa_circ(G, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]).draw(filename="../latex/img/circuits/qiskit_circuit_p-3_1.0.png")
create_qaoa_circ(G, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0]).draw(filename="../latex/img/circuits/qiskit_circuit_p-3_2.0.png")

In [ ]:
def maxcut_obj(x, G):
    obj = 0
    for i, j in G.edges():
        if x[i] != x[j]:
            obj -= 1
    return obj

def compute_expectation(counts, G):
    avg = 0
    sum_count = 0

    for bitstr, count in counts.items():
        obj = maxcut_obj(bitstr[::-1], G)
        avg += obj * count
        sum_count += count

    return avg/sum_count

def create_qaoa_circ(G, theta):
    nqubits = len(G.nodes())
    p = len(theta)//2 # num layers
    qc = QuantumCircuit(nqubits)

    beta = theta[:p]
    gamma = theta[p:]

    # |v0>
    for i in range(0, nqubits):
        qc.h(i)

    for irep in range(0, p):
        qc.barrier()

        # Hp
        for pair in list(G.edges()):
            qc.rzz(2 * gamma[irep], pair[0], pair[1])

        qc.barrier()

        # Hm
        for i in range (0, nqubits):
            qc.rx(2 * beta[irep], i)

    qc.measure_all()

    return qc

def get_expectation(G, shots=512):
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots

    def execute_circ(theta):
        qc = create_qaoa_circ(G, theta)
        counts = backend.run(qc, nshots=512).result().get_counts()
        return compute_expectation(counts, G)

    return execute_circ

In [ ]:
from scipy.optimize import minimize

expectation = get_expectation(G)

num_layers = 1
res = minimize(expectation, [1.0, 1.0] * num_layers, method = "COBYLA")
res

In [ ]:
from qiskit.visualization import plot_histogram

backend = Aer.get_backend('aer_simulator')
backend.shots = 512

qc_res = create_qaoa_circ(G, res.x)

counts = backend.run(qc_res, seed_simulator=10).result().get_counts()
plot_histogram(counts)